In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from datetime import datetime
import requests
import pandas_datareader as pdr
from datetime import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# Function to Validate Tickers
def validateTick(tick):
    
    # Try Inputting into Yahoo Stocks
    try:
        pdr.DataReader(f'{tick}','yahoo',date)
    
    # Raise Error if Invalid
    except:
        raise ValueError("Ticker Not Valid.")
        
# Function to Validate Date
def validate(date_text):
    
    # Try Check on Format
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        
# Function to Validate Date
def validateShift(shift):
    shift = int(shift)
    # Try Check on Format
    try:
        if 0 > shift <= 365:
            pass
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Please enter a whole number.")

In [3]:
# Pick a Start Date
date = input("Pick a start date for your analysis (YYYY-MM-DD): ")

# Validate Correct Date Format
validate(date)

Pick a start date for your analysis (YYYY-MM-DD):  2016-01-01


In [5]:
# Pick a Stock
stock_selection = input("Pick a stock ticker for your analysis: ")

# Validate Correct Date Format
validateTick(stock_selection)

Pick a stock ticker for your analysis:  CME


In [6]:
# Pick Shift
shift_selection = input("Pick how many days to shift your close values for your analysis: ")

# Validate Correct Date Format
validateShift(shift_selection)

Pick how many days to shift your close values for your analysis:  1


In [7]:
# Function to Create Stocks
def stockCreation(date, stock_selection, days_shifted):
    
    # Read Stock Data
    globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
    # Create Shifted Close Column
    globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
    # Drop Rows Without Shift Values
    globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
    # Empty List to Hold Profit or Loss
    profit_loss = []
    
    # Iterate Through DataFrame
    for x in range(len(globals()[stock_selection])):
        
        # Handle Profitable Days
        if globals()[stock_selection]["Shifted Close"][x] > globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Profit" to Empty List
            profit_loss.append("Profit")
            
        # Handle Loss Days
        elif globals()[stock_selection]["Shifted Close"][x] < globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Loss" to Empty List
            profit_loss.append("Loss")
            
        # Handle Push Days
        elif globals()[stock_selection]["Shifted Close"][x] == globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Push" to Empty List
            profit_loss.append("Push")
    
    # Create Profit/Loss Column
    globals()[stock_selection]["Profit/Loss"] = profit_loss
    
    # Create Returns Column
    globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
    # Iterate to Create STD Columns
    for x in range(5, 20, 5):
        
        # Create STD Column Based on Iteration
        col_name = 'std_' + str(x)
        
        # Add STD Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

    # Iterate to Create MA Columns
    for x in range(10, 30, 5):
        
        # Create MA Column Based on Iteration
        col_name = 'ma_' + str(x)
        
        # Add MA Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

    # Iterate to Create % Change Columns
    for x in range(3, 12, 3):
        
        # Create % Change Column Based on Iteration
        col_name = 'pct_' + str(x)
        
        # Add % Change Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

    # Create VMA 4 Column
    col_name = 'vma_4'
    
    # Add VMA 4 Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

    # Create Intraday Column
    col_name = 'co'
    
    # Add Intraday Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
    # Drop All NaN's
    globals()[stock_selection].dropna(inplace=True)
    
    # Return DataFrame
    return globals()[stock_selection]

In [8]:
# ### Function to Create Stocks
# def stockCreation(date, stock_selection, days_shifted):
    
#     # Read Stock Data
#     globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
#     # Create Shifted Close Column
#     globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
#     # Drop Rows Without Shift Values
#     globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
#     # Empty List to Hold Profit or Loss
#     profit_loss = []
    
#     # Iterate Through DataFrame
#     for x in range(len(globals()[stock_selection])):
        
#         # Save Adjusted Close Value
#         adj_close = globals()[stock_selection]["Adj Close"][x]
        
#         # Save Shifted Close Value
#         shifted_close = globals()[stock_selection]["Shifted Close"][x]
        
#         # Handle Returns 3x as Large as Open
#         if shifted_close > (3 * adj_close):
            
#             # Append Number 8
#             profit_loss.append(8)
            
#         # Handle Returns within 2x and 3x as Large as Open
#         elif (3 * adj_close) <= shifted_close > (2 * adj_close):
            
#             # Append Number 7
#             profit_loss.append(7)
            
#         # Handle Returns within 1.75x and 2x as Large as Open
#         elif (2 * adj_close) <= shifted_close > (1.75 * adj_close):
            
#             # Append Number 6 
#             profit_loss.append(6)
        
#         # Handle Returns within 1.5x and 1.75x as Large as Open
#         elif (1.75 * adj_close) <= shifted_close > (1.5 * adj_close):
            
#             # Append Number 5
#             profit_loss.append(5)
            
#         # Handle Returns within 1,25x and 1.5x as Large as Open
#         elif (1.5 * adj_close) <= shifted_close > (1.25 * adj_close):
            
#             # Append Number 4
#             profit_loss.append(4)
            
#         # Handle Returns within 1.1x and 1.25x as Large as Open
#         elif (1.25 * adj_close) <= shifted_close > (1.1 * adj_close):
            
#             # Append Number 3
#             profit_loss.append(3)
            
#         # Handle Returns within 1.05x and 1.1x as Large as Open
#         elif (1.1 * adj_close) <= shifted_close > (1.05 * adj_close):
            
#             # Append Number 2
#             profit_loss.append(2)
            
#         # Handle Returns within 1x and 1.05x as Large as Open
#         elif (1.05 * adj_close) <= shifted_close > (1 * adj_close):
            
#             # Append Number 1
#             profit_loss.append(1)
            
#         # Handle Returns within .95x and 1x as Small as Open
#         elif (1 * adj_close) <= shifted_close > (.95 * adj_close):
            
#             # Append Number 0
#             profit_loss.append(0)
        
#         # Handle Returns within .9x and .95x as Small as Open
#         elif (.95 * adj_close) <= shifted_close > (.9 * adj_close):
            
#             # Append Number -1
#             profit_loss.append(-1)
            
#         # Handle Returns within .75x and .9x as Small as Open
#         elif (.9 * adj_close) <= shifted_close > (.75 * adj_close):
            
#             # Append Number -2
#             profit_loss.append(-2)
            
#         # Handle Returns within .5x and .75x as Small as Open
#         elif (.75 * adj_close) <= shifted_close > (.5 * adj_close):
            
#             # Append Number -3
#             profit_loss.append(-3)
            
#         # Handle Returns within .25x and .5x as Small as Open
#         elif (.5 * adj_close) <= shifted_close > (.25 * adj_close):
            
#             # Append Number -4
#             profit_loss.append(-4)
            
#         # Handle Returns within .1x and .25x as Small as Open
#         elif (.25 * adj_close) <= shifted_close > (.1 * adj_close):
            
#             # Append Number -5
#             profit_loss.append(-5)
            
#         # Handle Returns within .05x and .1x as Small as Open
#         elif (.1 * adj_close) <= shifted_close > (.05 * adj_close):
            
#             # Append Number -6
#             profit_loss.append(-6)
        
#         # Handle Returns Smaller than .05x as Small as Open
#         elif shifted_close < (.05 * adj_close):
            
#             # Append Number -7
#             profit_loss.append(-7)
    
#     # Create Profit/Loss Column
#     globals()[stock_selection]["Profit/Loss"] = profit_loss
    
#     # Create Returns Column
#     globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
#     # Iterate to Create STD Columns
#     for x in range(5, 20, 5):
        
#         # Create STD Column Based on Iteration
#         col_name = 'std_' + str(x)
        
#         # Add STD Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

#     # Iterate to Create MA Columns
#     for x in range(10, 30, 5):
        
#         # Create MA Column Based on Iteration
#         col_name = 'ma_' + str(x)
        
#         # Add MA Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

#     # Iterate to Create % Change Columns
#     for x in range(3, 12, 3):
        
#         # Create % Change Column Based on Iteration
#         col_name = 'pct_' + str(x)
        
#         # Add % Change Column to DataFrame
#         globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

#     # Create VMA 4 Column
#     col_name = 'vma_4'
    
#     # Add VMA 4 Column to DataFrame
#     globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

#     # Create Intraday Column
#     col_name = 'co'
    
#     # Add Intraday Column to DataFrame
#     globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
#     # Drop All NaN's
#     globals()[stock_selection].dropna(inplace=True)
    
#     # Return DataFrame
#     return globals()[stock_selection]

In [9]:
# Create Stock
stockCreation(date, stock_selection, shift_selection)

,High,Low,Open,Close,Volume,Adj Close,Shifted Close,Profit/Loss,Returns,std_5,...,std_15,ma_10,ma_15,ma_20,ma_25,pct_3,pct_6,pct_9,vma_4,co
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-08,90.669998,87.000000,88.410004,90.190002,2758100.0,75.330887,77.936836,Profit,0.018351,1.218740,...,2.022289,73.756442,72.653080,72.354897,72.608648,0.009896,0.004898,0.044908,2910525.0,-13.079117
2016-02-09,93.809998,89.050003,89.050003,93.309998,3233500.0,77.936836,75.614868,Loss,0.034008,2.126921,...,2.423534,74.336102,73.053999,72.620506,72.763668,0.078829,0.044054,0.075449,3200800.0,-11.113167
2016-02-10,93.489998,90.440002,93.029999,90.529999,2133700.0,75.614868,74.604225,Loss,-0.030246,2.154174,...,2.382844,74.654331,73.406474,72.757486,72.802423,0.023321,0.017951,0.041389,2785025.0,-17.415131
2016-02-11,90.550003,88.320000,88.360001,89.320000,1952800.0,74.604225,76.132721,Profit,-0.013456,1.512457,...,1.967143,74.840592,73.814632,72.972562,72.815453,-0.008565,0.001331,-0.003668,2519525.0,-13.755775
2016-02-12,91.290001,89.510002,90.919998,91.150002,1691200.0,76.132721,75.765213,Loss,0.020281,1.253418,...,1.863000,74.949174,74.169889,73.241928,72.936396,-0.022671,0.056159,0.021383,2252800.0,-14.787277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-04,194.839996,192.160004,192.940002,193.529999,1394300.0,193.529999,191.389999,Loss,0.007989,4.823831,...,4.919194,186.012999,187.621333,190.559499,188.921600,0.032298,0.080244,0.051681,1475575.0,0.589996
2021-02-05,194.449997,188.429993,194.089996,191.389999,1741400.0,191.389999,191.740005,Profit,-0.011119,2.283465,...,4.705500,186.748000,187.437999,190.188499,189.457200,-0.003141,0.048603,0.038830,1515250.0,-2.699997
2021-02-08,193.669998,190.240005,192.940002,191.740005,2071100.0,191.740005,191.970001,Profit,0.001827,0.820621,...,4.700221,187.485001,187.432666,189.791500,189.844800,-0.001208,0.054433,0.044022,1625900.0,-1.199997


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-06-01,100.019997,100.010002,100.019997,100.010002,1600.0,99.960022
2020-06-02,100.019997,100.013000,100.019997,100.013000,4000.0,99.963020
2020-06-03,100.014999,100.014999,100.014999,100.014999,100.0,99.965019
2020-06-04,100.021004,100.000000,100.000000,100.014999,2400.0,99.965019
2020-06-05,100.019997,100.010002,100.019997,100.010002,400.0,99.960022
...,...,...,...,...,...,...
2021-02-05,100.019997,100.010002,100.019997,100.014999,71500.0,100.014999
2021-02-08,100.019997,100.010002,100.019997,100.014999,106700.0,100.014999
2021-02-09,100.019997,100.010002,100.019997,100.010002,34100.0,100.010002


In [10]:
# Assign Y Values
y = stockCreation(date, stock_selection, shift_selection)["Profit/Loss"]

In [11]:
# Assign X Values
X = stockCreation(date, stock_selection, shift_selection).drop(["Shifted Close", "Profit/Loss"], axis=1)

In [12]:
# Train/Test Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

array([[0.13820115, 0.13996428, 0.12854723, ..., 0.67744714, 0.08481401,
        0.24041312],
       [0.81723453, 0.81759977, 0.81572934, ..., 0.57139129, 0.0909769 ,
        0.35354594],
       [0.73335302, 0.74084268, 0.73980981, ..., 0.59340711, 0.51037578,
        0.28174684],
       ...,
       [0.54652279, 0.55933592, 0.54662051, ..., 0.55665483, 0.34610158,
        0.63572399],
       [0.84480082, 0.85556873, 0.83880006, ..., 0.71716128, 0.03833001,
        0.43530384],
       [0.57445863, 0.57184331, 0.56806953, ..., 0.60311603, 0.24372807,
        0.59541555]])

In [ ]:
X_train = []
y_train = []

for i in range 